In [ ]:
from datetime import datetime, date
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Constants
INPUT_DATA_PATH = "/kaggle/input/ashrae-energy-prediction"

MIN_TRAIN_TIMESTAMP = pd.Timestamp("2016-01-01 00:00:00")
MAX_TRAIN_TIMESTAMP = pd.Timestamp("2016-12-31 23:00:00")
TRAIN_DATA_RESOLUTION = "1h"

WEATHER_FEATURE_COLUMNS = [
    'air_temperature',
    'cloud_coverage',
    'dew_temperature',
    'precip_depth_1_hr',
    'sea_level_pressure',
    'wind_direction',
    'wind_speed'
]

In [ ]:
# Load raw data
# train_df = pd.read_csv(f"{INPUT_DATA_PATH}/train.csv")
# building_df = pd.read_csv(f"{INPUT_DATA_PATH}/building_metadata.csv")
weather_df = pd.read_csv(f"{INPUT_DATA_PATH}/weather_train.csv")

In [ ]:
# Process weather data

def reindex_weather_data(weather_df: pd.DataFrame) -> pd.DataFrame:
    weather_df["timestamp"] = pd.to_datetime(weather_df["timestamp"])

    # Reindex weather data
    timestamps = pd.date_range(
        MIN_TRAIN_TIMESTAMP,
        MAX_TRAIN_TIMESTAMP,
        freq=TRAIN_DATA_RESOLUTION,
        inclusive="both"
    )
    timestamps = pd.DatetimeIndex(timestamps, name="timestamp")
    site_dfs = []
    for site_id, site_df in weather_df.groupby("site_id", observed=True):
        site_df = site_df.set_index("timestamp").reindex(timestamps).reset_index()
        site_df["site_id"] = site_df["site_id"].fillna(value=site_id)
        site_dfs.append(site_df)

    weather_df = pd.concat(site_dfs, ignore_index=True)
    return weather_df


def interpolate(weather_df: pd.DataFrame, column: str) -> pd.DataFrame:
    weather_df[column] = weather_df[column].interpolate("linear", limit=12)
    weather_df[column] = weather_df[column].ffill(limit=2)
    weather_df[column] = weather_df[column].bfill(limit=2)
    return weather_df


def _mean_weather_by_date_and_site(weather_df, column) -> pd.DataFrame:
    mean_values = (
        weather_df
        .groupby(["date", "site_id"])[[column]]
        .mean()
        .reset_index()
    )
    return mean_values


def _merge_onto_weather_df(weather_df, right, right_suffix) -> pd.DataFrame:
    weather_df = weather_df.merge(
        right=right,
        how="left",
        on=["date", "site_id"],
        suffixes=("", right_suffix)
    )
    return weather_df


def impute_with_same_day_mean(weather_df: pd.DataFrame, column: str) -> pd.DataFrame:
    # Compute same day mean values and merge onto weather df
    mean_values = _mean_weather_by_date_and_site(weather_df, column)
    weather_df = _merge_onto_weather_df(weather_df, mean_values, "_mean")
    
    # Fill with means from same day
    weather_df[column] = weather_df[column].fillna(weather_df[f"{column}_mean"])
    weather_df = weather_df.drop(f"{column}_mean", axis=1)
    return weather_df


def ffill_mean_by_date(weather_df: pd.DataFrame, column: str) -> pd.DataFrame:
    # Compute same day mean values and merge onto weather df
    mean_values = _mean_weather_by_date_and_site(weather_df, column)
    
    # ffill by site and date
    site_dfs = []
    for site_id, site_df in mean_values.groupby("site_id"):
        site_df = site_df.sort_values("date")
        site_df[column] = site_df[column].ffill().bfill()
        site_dfs.append(site_df)
    mean_values = pd.concat(site_dfs, ignore_index=True)
    
    # Merge back onto main and fill with mean values
    weather_df = _merge_onto_weather_df(weather_df, mean_values, "_mean")
    weather_df[column] = weather_df[column].fillna(weather_df[f"{column}_mean"])
    weather_df = weather_df.drop(f"{column}_mean", axis=1)
    return weather_df


def fill_missing_weather_data(weather_df: pd.DataFrame) -> pd.DataFrame:
    weather_df["date"] = weather_df["timestamp"].dt.date

    for column in WEATHER_FEATURE_COLUMNS:
        weather_df = interpolate(weather_df, column)
        weather_df = impute_with_same_day_mean(weather_df, column)
        weather_df = ffill_mean_by_date(weather_df, column)
    
    return weather_df

In [ ]:
def missing_weather_stats(weather_df: pd.DataFrame) -> pd.DataFrame:
    nan_count = weather_df.isna().sum().to_frame("n_missing")
    nan_count["pct_missing"] = nan_count["n_missing"] / len(weather_df) * 100
    return nan_count

In [ ]:
weather_df = reindex_weather_data(weather_df)
missing_weather_stats(weather_df)

In [ ]:
weather_df = fill_missing_weather_data(weather_df)
missing_weather_stats(weather_df)